In [ ]:
%pip install openai langchain google-api-python-client openai"[embeddings]"

In [45]:
# Global config
generate_embeddings = False
agent_verbose = True

In [9]:
# Generate Sample-Faq Embeddings

from openai.embeddings_utils import get_embedding
import pandas as pd

if generate_embeddings:
    df = pd.read_csv("data/Sample-Faq.csv", na_filter=False)

    df["Combined"] = df.Query.str.strip() + "," + df.Question.str.strip()

    df["Embedding"] = df.Combined.apply(
        lambda x: get_embedding(x, engine="embedding-ada")
    )

    df.to_csv("data/Sample-Faq-Embeddings.csv", index=False)

    df.head(2)

,Query,Question,Answer,Combined,Embedding
0,"Ordering process,Ordering instructions",How can I order?,You can order easily using our online platform...,"Ordering process,Ordering instructions,How can...","[0.0078082336112856865, -0.011418634094297886,..."
1,"Online ordering benefits,Online ordering discount",Why should I buy online?,Speeding up the process. By ordering online yo...,"Online ordering benefits,Online ordering disco...","[0.0011960654519498348, -0.017113571986556053,..."


In [ ]:
# Generate a dataset for products

import openai

if generate_embeddings:
    max_products = 30
    prompt = f"""
    Generate a dataset for products with the following requirements:
    1. Categories are Toys, Apparel, Electronics, Kitchen appliances and Home decor.
    2. Each product should have an ID, Name, Description (200 characters max) and Category.
    3. Toys should have an extra range of age group associated with each product info. This is an optional for all other categories.
    4. The dataset must have {max_products} products.
    5. The product description should be wrapped in double quotes.
    6. The dataset must be in CSV format.
    """

    response = openai.Completion.create(
        engine="testdavinci003",
        prompt=prompt,
        temperature=1,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )

    print(response.choices[0].text)

In [31]:
# Generate Products Embeddings

from openai.embeddings_utils import get_embedding
import pandas as pd

if generate_embeddings:
    df = pd.read_csv("data/Products.csv", na_filter=False)

    df["Combined"] = (
        df.ID.astype("str")
        + ","
        + df.Name.str.strip()
        + ","
        + df.Description.str.strip()
        + ","
        + df.Category.str.strip()
        + ","
        + df.Age.str.strip()
    )

    df["Embedding"] = df.Combined.apply(
        lambda x: get_embedding(x, engine="embedding-ada")
    )

    df.to_csv("data/Products-Embeddings.csv", index=False)

    df.head(2)

In [58]:
import os
import redis

from redis.commands.search.field import VectorField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

# Redis connection details
redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_password = os.getenv("REDIS_PASSWORD")

# Connect to the Redis server
conn = redis.Redis(
    host=redis_host,
    port=redis_port,
    password=redis_password,
    encoding="utf-8",
    decode_responses=True,
)

p = conn.pipeline(transaction=False)

In [59]:
# Store Sample-Faq Embeddings

import numpy as np
import pandas as pd
import uuid


def store_faq_embedding(key, question, answer, embedding):
    # Convert to numpy array and bytes
    vector = np.array(eval(embedding)).astype(np.float32).tobytes()

    # Create a new hash with url and embedding
    text = f"{question} {answer}"
    post_hash = {key: text, "embedding": vector}

    # Create hash
    conn.hset(name=f"{key}:{uuid.uuid1()}", mapping=post_hash)


faq_key = "faq"
faq_index_name = "faq-index"
df = pd.read_csv("data/Sample-Faq-Embeddings.csv", na_filter=False)
df.apply(
    lambda x: store_faq_embedding(faq_key, x.Question, x.Answer, x.Embedding),
    axis=1,
)
p.execute()

SCHEMA = [
    TextField(faq_key),
    VectorField(
        "embedding",
        "HNSW",
        {"TYPE": "FLOAT32", "DIM": 1536, "DISTANCE_METRIC": "COSINE"},
    ),
]

try:
    conn.ft(faq_index_name).create_index(
        fields=SCHEMA,
        definition=IndexDefinition(prefix=[f"{faq_key}:"], index_type=IndexType.HASH),
    )

    print(f"Created {faq_index_name} index")
except Exception as e:
    print("Index already exists")

Created faq-index index


In [60]:
# Store Products Embeddings


def store_product_embedding(key, id, name, desc, category, age, embedding):
    # Convert to numpy array and bytes
    vector = np.array(eval(embedding)).astype(np.float32).tobytes()

    # Create a new hash with url and embedding
    text = f'{id},{name},"{desc}",{category},{age}'
    post_hash = {key: text, "embedding": vector}

    # Create hash
    conn.hset(name=f"{key}:{uuid.uuid1()}", mapping=post_hash)


product_key = "product"
product_index_name = "product-index"
df = pd.read_csv("data/Products-Embeddings.csv", na_filter=False)
df.apply(
    lambda x: store_product_embedding(
        product_key, x.ID, x.Name, x.Description, x.Category, x.Age, x.Embedding
    ),
    axis=1,
)
p.execute()

SCHEMA = [
    TextField(product_key),
    VectorField(
        "embedding",
        "HNSW",
        {"TYPE": "FLOAT32", "DIM": 1536, "DISTANCE_METRIC": "COSINE"},
    ),
]

try:
    conn.ft(product_index_name).create_index(
        fields=SCHEMA,
        definition=IndexDefinition(
            prefix=[f"{product_key}:"], index_type=IndexType.HASH
        ),
    )

    print(f"Created {product_index_name} index")
except Exception as e:
    print("Index already exists")

Created product-index index


In [71]:
# Integrate redis search

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.tools import tool

from openai.embeddings_utils import get_embedding
from redis.commands.search.query import Query


def search_vector(index, text_field, text, client, top_k=3):
    vector = get_embedding(text, engine="embedding-ada")
    query_vector = np.array(vector).astype(np.float32).tobytes()
    base_query = f"*=>[KNN {top_k} @embedding $vector AS vector_score]"
    query = (
        Query(base_query)
        .return_fields(text_field, "vector_score")
        .sort_by("vector_score")
        .dialect(2)
    )

    try:
        results = client.ft(index).search(query, query_params={"vector": query_vector})
    except Exception as e:
        print("Error calling Redis search: ", e)
        return None

    return results


@tool
def search_faq(query: str) -> str:
    """Search for frequently asked questions (FAQ)."""
    results = search_vector("faq-index", "faq", query, conn)
    faq_results = ""
    id = 1
    for result in results.docs:
        faq_results += f"{id}. {result.faq}\n"
        id += 1

    return faq_results


@tool
def search_product(query: str) -> str:
    """Search for products"""
    results = search_vector("product-index", "product", query, conn, 30)
    product_results = ""
    for result in results.docs:
        product_results += f"{result.product}\n"

    prompt = f"""
```csv
ID,Name,Description,Category,Suitable Age
{product_results}
```
"""

    return prompt


chat = AzureChatOpenAI(temperature=1, deployment_name="gpt-35-turbo")
llm = AzureOpenAI(temperature=0, deployment_name="testdavinci003")
tools = load_tools(["llm-math"], llm=llm) + [search_faq, search_product]

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=agent_verbose,
    # https://github.com/hwchase17/langchain/issues/1358
    handle_parsing_errors="Check your output and make sure it conforms!",
)

agent.run(
    """
You are a friendly AI assistant. When a user is asking for any recommendation about products, always use the age info provided by the tools. Answer the following question.

What are suitable toys for a 9 years old?"""
)



> Entering new  chain...
Thought: I can use the `search_product` tool to find recommended toys for a 9-year-old based on their age.

Action:
```
{
  "action": "search_product",
  "action_input": "toys for 9 year olds"
}
```


Observation: 
```csv
ID,Name,Description,Category,Suitable Age
3,Bicycle,"A two-wheeled pedal-driven vehicle",Toys,From 7 to 11
4,Train Set,"A set of miniature trains and tracks",Toys,From 7 to 11
24,Puzzle Game,"A fun game which requires solving a problem or puzzle",Toys,From 4 to 7
23,Toy Robot,"A toy version of an artificial humanoid robot",Toys,From 8 to 11
13,Remote Control Car,"A toy car, which is operated by a TV remote control",Toys,From 6 to 8
14,Lego Set,"Interlocking pieces of plastic or other material used to build creations",Toys,From 4 to 7
1,Action Figure,"A figurine of a superhero equipped with weapons and articulation",Toys,From 6 to 8
2,Stuffed Animal,"A soft toy made from synthetic fabric",Toys,From 3 to 5
9,Toaster,"A kitchen appliance used t

'The suitable toys for a 9-year-old are bicycles, train sets, and toy robots.'

In [ ]:
# Set up Agent

from langchain.agents import AgentExecutor
from langchain.agents.loading import AGENT_TO_CLASS
from langchain.memory import ConversationBufferMemory

# Define system message
system_message = """You are a friendly AI assistant. You help answer questions from users about our online store that sells many products.

You have access to the following tools:

1. search_product: useful to search for products.
2. search_faq: useful to search for answers about order, payment, delivery, return policy and customer services.
3. llm-math: useful to parse CSV format.

Please always try to use the tools to find answers, do not use any information that isn't coming from the tools.

Before using any tool, remember to check the previous messages to see if you can find the answer.

If the user is complaining about our products, follow this process:
    1. Ask politely about the issue.
    2. Gather details from them about the product such as order id.
    3. Try to find out if the complaint is about technical aspects or delivery related.
    4. Confirm user that the complaint is registered and referred to relevant department (Technicals / Delivery).
    5. Sign off as "Contoso AI Agent - CAIA".

If the user wants to see what products are available, follow these steps:
    1. Use the `search_product` tool to retrieve the list of products and store it in a product_list array.
    2. Show the list of products to the user in CSV format.

If the user wants to buy a product, search for it in the product_list array and show it to the user for confirmation. If the user has confirmed to buy that product, add it to a cart array.

If the user wants to checkout, show all items in the cart array and ask for confirmation. If the user has confirmed to checkout, show the receipt in JSON format.
"""

# Define format instructions
# Workaround #1 for the `Could not parse LLM output` issue:
# https://github.com/hwchase17/langchain/issues/1358#issuecomment-1569741405
format_instructions = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the following format(the prefix of "Thought: " and "{ai_prefix}: " are must be included):

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""

llm = AzureChatOpenAI(temperature=0.2, deployment_name="gpt-35-turbo")

agent_cls = AGENT_TO_CLASS[AgentType.CONVERSATIONAL_REACT_DESCRIPTION]

agent_obj = agent_cls.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prefix=system_message,
    format_instructions=format_instructions,
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent_obj,
    tools=tools,
    memory=memory,
    verbose=agent_verbose,
)

In [ ]:
# Set up Panel

import panel as pn

from langchain.schema import (
    AIMessage,
    HumanMessage,
    OutputParserException,
)

# Panel
pn.extension(loading_spinner="dots", loading_color="#00aa41")
inp = pn.widgets.TextInput(value="", placeholder="Enter text here...")
button_conversation = pn.widgets.Button(name="Chat!")
convos = []  # store all panel objects in a list


def get_conversations(_):
    prompt = inp.value
    inp.value = ""
    if prompt != "":
        # Workaround #2 for the `Could not parse LLM output` issue.
        # Sometimes, the model still doesn't follow our given instructions so we need this
        # as a backup solution:
        # https://github.com/hwchase17/langchain/issues/1358#issuecomment-1486132587
        try:
            openai_answer = agent_executor(prompt)["output"]
        except OutputParserException as e:
            response = str(e)
            print(f"OutputParserException: {response}")
            response = response.removeprefix(
                "Could not parse LLM output: `"
            ).removesuffix("`")
            memory.chat_memory.add_message(HumanMessage(content=prompt))
            memory.chat_memory.add_message(AIMessage(content=response))
            openai_answer = response

        convos.append(pn.Row("\U0001F60A", pn.pane.Markdown(prompt, width=600)))
        convos.append(
            pn.Row(
                "\U0001F916",
                pn.pane.Markdown(
                    openai_answer, width=600, styles={"background-color": "#F6F6F6"}
                ),
            )
        )

    return pn.Column(*convos)


interactive_conversation = pn.bind(get_conversations, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=500),
)

dashboard.servable()